In [ ]:
#! pip3 install telethon
# ! pip3 install nest_asyncio
# https://my.telegram.org/apps


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
from telethon.sync import TelegramClient
from telethon.tl.types import MessageMediaPhoto
from datetime import datetime
import nest_asyncio
import os
import json

nest_asyncio.apply()
from dotenv import load_dotenv
load_dotenv()

import os
api_id = os.getenv("TELEGRAM_API_ID")
api_hash = os.getenv("TELEGRAM_API_HASH")
client = TelegramClient('session', api_id, api_hash)

async def scrape_channel(channel_username, limit=50):
    await client.start()
    print(f"Scraping messages from {channel_username}")
    
    messages = await client.get_messages(channel_username, limit=limit)
    os.makedirs(f"media/{channel_username}", exist_ok=True)
    
    data = []
    
    for i, message in enumerate(messages):
        msg_data = {
            "text": message.text or "",
            "date": str(message.date),
            "sender_id": message.sender_id,
            "has_media": bool(message.media),
            "media_path": None
        }
        if message.media and isinstance(message.media, MessageMediaPhoto):
            file_path = await message.download_media(file=f"media/{channel_username}/")
            msg_data["media_path"] = file_path
            print(f"Saved image to: {file_path}")
        
        print(f"{i+1}. {msg_data['text'][:50]}")
        data.append(msg_data)

    with open(f"data/{channel_username}_messages.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

    print(f"✅ Saved {len(data)} messages to JSON.")
    await client.disconnect()
